# OMERO Image Annotation Workflow with micro-SAM
This notebook guides you through annotating images in OMERO using the micro-SAM annotation plugin. Each step is explained clearly.

## 1. Setup and Installation
Install and import the required libraries. Make sure you have access to OMERO and the omero-annotate-ai package.

In [1]:
# Import the main package and check dependencies
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
 )

import os
from pathlib import Path

print(f"omero-annotate-ai version: {omero_annotate_ai.__version__}")
try:
    import ezomero
    print("OMERO functionality: Available")
except ImportError:
    print("OMERO functionality: Not available. Install with: pip install -e .[omero]")

omero-annotate-ai version: 0.1.0
OMERO functionality: Available


## 2. Connect to OMERO
Use the widget below to connect to your OMERO server. Fill in your server details and test the connection before proceeding.

In [2]:
# Create and display the OMERO connection widget
print("OMERO Connection Setup")
conn_widget = create_omero_connection_widget()
conn_widget.display()
print("\nAfter connecting, run the next cell to continue.")

OMERO Connection Setup
Loaded configuration from connection history: root@lcows1



After connecting, run the next cell to continue.


In [6]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established. Please use the widget above to connect.")

print("OMERO connection established.")

OMERO connection established.


## 3. Configure the Annotation Workflow
Set up your annotation workflow using the widget below. Select your working directory, OMERO container, and configure annotation parameters.

In [ ]:
# Create and display the workflow widget
print("Annotation Workflow Setup")
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()
print("\nAfter completing the workflow, run the next cell.")

Annotation Workflow Setup



After completing the workflow, run the next cell.


## 4. Review and Run the Annotation Pipeline
Preview your configuration, validate your OMERO selection, and run the annotation pipeline.

#### 4.1. Configuration summary

In [8]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

print("Configuration Summary:")
print(f"Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"Training Set: {config.name}")
print(f"Model: {config.ai_model.model_type}")
print(f"Channel: {config.spatial_coverage.channels}")
print(f"Output: {config.output.output_directory}")
print(f"Resume from Table: {config.workflow.resume_from_table}")
print(f"Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})")

if config.spatial_coverage.three_d:
    print("3D processing: Enabled")

Configuration Summary:
Container: project (ID: 52)
Training Set: test_20251013_110205
Model: vit_b_lm
Channel: [0]
Output: /var/home/maartenpaul/omero_annotate_ai/omero_annotations
Resume from Table: False
Read-only Mode: False
Patches: 1 per image (512×512)


#### 4.2. Convert configuration to an annotation pipeline

In [9]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"Validating {container_type} with ID {container_id}...")
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")
print(f"Found {container_type}: {container.getName()}")

Validating project with ID 52...
Found project: test


#### 4.3. Run the annotation pipeline on the files
After loading of the files the napari will open for you to annotate the images

In [10]:
# Run the complete annotation pipeline using the updated full workflow function
print("Starting annotation pipeline...")
print(f"Using micro-SAM model: {config.ai_model.model_type}")

if config.processing.batch_size == 0:
    print("Processing: All images in one batch")
else:
    print(f"Processing: Batches of {config.processing.batch_size} images")

print("Napari will open for interactive annotation.")

try:
    # Run the complete micro-SAM workflow (setup, schema, table, annotation)
    table_id, updated_config = pipeline.run_full_micro_sam_workflow()
    print("Annotation pipeline completed successfully.")
    print(f"Tracking table ID: {table_id}")
    if updated_config.workflow.read_only_mode:
        print(f"Annotations saved locally to: {updated_config.output.output_directory}")
    else:
        print("Annotations uploaded to OMERO")
    print(f"Total images processed: {len(updated_config.get_processed())}")
except Exception as e:
    print(f"Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()

Starting annotation pipeline...
Using micro-SAM model: vit_b_lm
Processing: All images in one batch
Napari will open for interactive annotation.
Loading image IDs from project 52
Found 288 image IDs
Loaded 6 images (by ID)
Defining annotation schema...
Schema defined: 6 annotations
Creating OMERO tracking table...
Created/replaced tracking table 'micro_sam_training_test_20251013_110205' with 6 units
   Container: project 52
   Table ID: 855
Created OMERO table with ID: 855
Running micro-SAM annotation on 6 unprocessed annotations

Processing batch 1/1
   Batch size: 6
   Config state before: 0/6 processed
   Running annotation...
Loading 1 images using dask...
Materializing dask arrays to numpy...
   Processing chunk 1/1
Successfully loaded 1 images
Loaded 2D data shape: (512, 512)
Loading 1 images using dask...
Materializing dask arrays to numpy...
   Processing chunk 1/1
Successfully loaded 1 images
Loaded 2D data shape: (512, 512)
Loading 1 images using dask...
Materializing dask ar

Precompute state for files: 100%|██████████| 6/6 [00:57<00:00,  9.59s/it]


Precomputation took 59.7476692199707 seconds (= 00:60 minutes)
The first image to annotate is image number 0


INFO: Computed embeddings for the 'Light Microscopy' model of size 'base'.
Loading next image: at index 1
Loading next image: at index 2
Loading next image: at index 3
Loading next image: at index 4
Loading next image: at index 5
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
File ~/Documents/GitHub/omero_annotate_ai/.pixi/envs/dev/lib/python3.11/site-packages/psygnal/_signal.py:1309, in SignalInstance._run_emit_loop(self=<class 'psygnal._signal.SignalInstance'> instance, args=(False,))
   1308     with Signal._emitting(self):
-> 1309         self._run_emit_loop_inner()
        Exception trying to inspect frame. No more locals available.   1310 except RecursionError as e:

File ~/Documents/GitHub/omero_annotate_ai/.pixi/envs/dev/lib/python3.11/site-packages/psygnal/_signal.py:1338, in SignalInstance._run_emit_loop_immediate(self=<class 'psygnal._signal.SignalInstance'> instance)
   

Traceback (most recent call last):
  File "/tmp/ipykernel_166237/75507828.py", line 14, in <module>
    table_id, updated_config = pipeline.run_full_micro_sam_workflow()
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/home/maartenpaul/Documents/GitHub/omero_annotate_ai/src/omero_annotate_ai/core/annotation_pipeline.py", line 1478, in run_full_micro_sam_workflow
    return self.run_full_microsam_workflow(images_list)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/home/maartenpaul/Documents/GitHub/omero_annotate_ai/src/omero_annotate_ai/core/annotation_pipeline.py", line 1474, in run_full_microsam_workflow
    return self.run_microsam_annotation()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/home/maartenpaul/Documents/GitHub/omero_annotate_ai/src/omero_annotate_ai/core/annotation_pipeline.py", line 1157, in run_microsam_annotation
    self._finalize_workflow(processed_count)
  File "/var/home/maartenpaul/Documents/GitHub/

#### Save the configuration file to the local folder

In [ ]:
# Export configuration for future use
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"Configuration saved to: {config_path}")
except Exception as e:
    print("Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"Configuration saved to: {config_filename}")

print("To reuse this configuration:")
print("from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")

## 5. Cleanup (Recommended)
Close the OMERO connection when you are finished. This is good practice, especially when working with shared servers.

In [10]:
# Close OMERO connection (optional, but recommended)
if 'conn' in locals() and conn is not None:
    conn.close()
    print("OMERO connection closed.")

OMERO connection closed.
